In [44]:
import pandas_datareader as pdr
import pandas as pd
from os import listdir
from os.path import isfile, join
from random import randint
from time import sleep
from numpy import random
from datetime import date, timedelta
import warnings
warnings.filterwarnings("ignore")

# Get IDX stock List

In [45]:
idx_stock_lists = pd.read_csv('idx_stock_lists_correct_date.csv', index_col='No')
idx_stock_lists.rename(columns={'Kode/Nama Perusahaan':'Ticker_Symbols'}, inplace=True)
idx_stock_lists.rename(columns={'Tanggal Pencatatan':'Date'}, inplace=True)
idx_stock_lists
#preview of 700 stock in IDX
#my database dont include all of this, only stocks that has record from 2016

,Ticker_Symbols,Nama,Date,Saham,Papan Pencatatan
No,,,,,
1,AALI,Astra Agro Lestari Tbk.,1997-12-09,1.924.688.333,Utama
2,ABBA,Mahaka Media Tbk.,2002-04-03,2.755.125.000,Pengembangan
3,ABDA,Asuransi Bina Dana Arta Tbk.,1989-07-06,620.806.680,Pengembangan
4,ABMM,ABM Investama Tbk.,2011-12-06,2.753.165.000,Utama
5,ACES,Ace Hardware Indonesia Tbk.,2007-11-06,17.150.000.000,Utama
...,...,...,...,...,...
696,YPAS,Yanaprima Hastapersada Tbk,2008-03-05,668.000.089,Pengembangan
697,YULE,Yulie Sekuritas Indonesia Tbk.,2004-12-10,1.785.000.000,Pengembangan
698,ZBRA,Zebra Nusantara Tbk,1991-08-01,856.133.009,Pengembangan


## Clean idx list dataframe

In [46]:
## Lets clean the dataframe a little bit
idx_stock_lists.dtypes #dtypes
#need to change tanggal pencatatan into pd datetime
#nned to change saham into int

Ticker_Symbols      object
Nama                object
Date                object
Saham               object
Papan Pencatatan    object
dtype: object

In [47]:
idx_stock_lists['Date'] = pd.to_datetime(idx_stock_lists['Date'])
idx_stock_lists['Saham'] = idx_stock_lists['Saham'].str.replace('.','')
idx_stock_lists['Saham'] = pd.to_numeric(idx_stock_lists['Saham'])
idx_stock_lists.head()#done

,Ticker_Symbols,Nama,Date,Saham,Papan Pencatatan
No,,,,,
1,AALI,Astra Agro Lestari Tbk.,1997-12-09,1924688333,Utama
2,ABBA,Mahaka Media Tbk.,2002-04-03,2755125000,Pengembangan
3,ABDA,Asuransi Bina Dana Arta Tbk.,1989-07-06,620806680,Pengembangan
4,ABMM,ABM Investama Tbk.,2011-12-06,2753165000,Utama
5,ACES,Ace Hardware Indonesia Tbk.,2007-11-06,17150000000,Utama


# Build Initial Database for every stock's historical price


In [48]:
#get stocks list
stock_list = list(idx_stock_lists['Ticker_Symbols'])

In [49]:
#get start date
#lets take from the earliest date
#pdr is smart, its only taking available dates for stocks
#so it doesnt matter the dates starts, it will takes the initial dates
start_date = '1980-01-01'

#get end date
#yesterday's date
yesterday = date.today() - timedelta(days=1)

error_no_fetched = [] # list of unable to  fetch stock's data, possible delisting like GREN
#we do this for future proof in case new stocks get into delisting list

for i in range(len(stock_list)):
    sleeptime = random.uniform(1, 3) #add sleeptime between 1 to 3 second so yahoo doesnt get suspicious
    sleep(sleeptime)
    try:
        temp_df = pdr.data.get_data_yahoo(stock_list[i]+'.JK', start_date, yesterday).reset_index() #yyyy-mm-dd 
        #the code above is taking specified stocks' data from its initial date to yesterday's date
        temp_df.to_csv('database\\' + stock_list[i] + '.csv', index=False) #save to temp database
    except:
        error_no_fetched.append(stock_list[i])
        continue
    


In [50]:
#check
pnbn = pd.read_csv('database//PNBN.csv')
pnbn

,Date,High,Low,Open,Close,Volume,Adj Close
0,2002-09-03,190.0,175.0,190.0,175.0,2684000.0,157.212784
1,2002-09-04,175.0,170.0,170.0,170.0,1231000.0,152.721008
2,2002-09-05,180.0,165.0,175.0,165.0,2637000.0,148.229187
3,2002-09-06,170.0,170.0,170.0,170.0,1020000.0,152.721008
4,2002-09-09,170.0,170.0,170.0,170.0,137500.0,152.721008
...,...,...,...,...,...,...,...
4688,2021-06-21,875.0,825.0,875.0,860.0,5837000.0,860.000000
4689,2021-06-22,860.0,835.0,860.0,835.0,3409200.0,835.000000
4690,2021-06-23,840.0,820.0,840.0,820.0,3038900.0,820.000000
4691,2021-06-24,830.0,815.0,820.0,820.0,2215500.0,820.000000


# Code Below to Update the database

In [52]:
#taking every stocks dataframe's name in our database and put them in a list
stocks_list_complete_name = [f for f in listdir('database') 
                             if isfile(join('database', f))]
stocks_list_complete_name[:5]

['AALI.csv', 'ABBA.csv', 'ABDA.csv', 'ABMM.csv', 'ACES.csv']

In [53]:
#separate the .csv part and taking their name only
stock_list_name_only = []
for i in stocks_list_complete_name:
    stock_list_name_only.append(i.split('.')[0]) #take every string, split each string by dot and take the first value of the index
stock_list_name_only[:5]

['AALI', 'ABBA', 'ABDA', 'ABMM', 'ACES']

In [56]:
#taking the each stock's last date in our dataframe
#put them in a dictionary
last_date = {}
for i in stock_list_name_only:
    a = pd.read_csv('database\\'+ i+'.csv')
    last_date[i] = list(a['Date'])[-1]
#last_date['PNBN'] check

'2021-06-25'

## Update from last date to yesterday's date

In [ ]:
#taking yesterday's date
yesterday_date = date.today() - timedelta(days=1)
corrected_format = yesterday_date.strftime("%Y-%m-%d")
corrected_format

## Get every stock's historical price from its last date to yesterday's date

In [ ]:
error_no_fetched = [] # list of unable to  fetch stock's data, possible delisting like GREN
#we do this for future proof in case new stocks get into delisting list

for i in stock_list_name_only:
    sleeptime = random.uniform(1, 3) #add sleeptime between 1 to 3 second so yahoo doesnt get suspicious
    sleep(sleeptime)
    try:
        temp_df = pdr.data.get_data_yahoo(i+'.JK', last_date[i], corrected_format).reset_index() #yyyy-mm-dd 
        #the code above is taking specified stocks' data from its last date to yesterday's date
        temp_df.to_csv('tmp\\' + i + '.csv', index=False) #save to temp database
    except:
        error_no_fetched.append(i)
        continue

## Concat the original database with newest date 

In [ ]:
new_list_stock_list_name_only = [i for i in stock_list_name_only if i not in error_no_fetched] #pick out error no fetch from out stocks name

for i in range(len(new_list_stock_list_name_only)):
    a = pd.read_csv('database\\' + new_list_stock_list_name_only[i] +'.csv') #take each stock original data frame in the database
    b = pd.read_csv('tmp\\' + new_list_stock_list_name_only[i] + '.csv')
    c = pd.concat([a, b]) #concat the dataframe into a
    c = c.drop_duplicates(subset='Date', keep='first') #experimental, to drop rows with same dates and keep first occurence
    c.reset_index(drop=True, inplace=True) #reset the index after concat
    c.to_csv('database\\' + new_list_stock_list_name_only[i] + '.csv', index=False) #save and at the sametime update our database
#it works 